In [1]:
###############
# Service version : 1.2.2
# Language        : Python/Jupyter

### Issues and To-Do Items

* The loop of keeping tell the exeuction status, it might be just get the old messgae without fresh?

* How to check the status after gain the job ID? -

    https://tenant1.predict.masinst1.ibmmam.com/ibm/modelfactory/service/log/$job_id

* To ensure the connection to the deployed site if you are outside of IBM network (or even in the network), us VPN at the CISCO client with this link - http://sasvpn.us.ibm.com/TUNNELALL



# Import Libraries & Functions

In [2]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [3]:
# Print functions
def print_job_status(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    status_data = get_response.json()
    
    # Print the job status
    if "status" in status_data:
        print("The status of job {} is {}.".format(job_id, status_data['status']))
    else:
        print(status_data)
    if status_data['status']:
        return status_data['status']
    else:
        return "initializing"
    
def print_all_status(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    status_data = get_response.json()
    
    # Print the job status
    print(status_data)



In [4]:
# More print functions using display 
from IPython.display import display, HTML
import requests

def print_job_details(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    summary_data = get_response.json()
    
    # Display the job status
    display(HTML(print_keys_and_values(summary_data)))
    return summary_data

    
def print_keys_and_values(json_data):
    # Start the HTML code
    html_code = "<div style='font-family: Arial; font-size: 1.2em;'>"
    
    # Add the job details to the HTML code
    html_code += f"<p>job_id: {json_data['job_id']}</p>"
    if "status" in json_data:
        html_code += f"<p>status: {json_data['status']}</p>"
    html_code += "<br>"
    if "detailed_summary" in json_data:
        for summary in json_data['detailed_summary']:
            html_code += f"<p>run_id: {summary['run_id']}</p>"
            html_code += f"<p>experiment_id: {summary['experiment_id']}</p>"
            if "status" in summary:
                print('summary',summary)
                html_code += f"<p>status: {summary['status']}</p>"
            html_code += f"<p>artifact_uri: {summary['artifact_uri']}</p>"
            html_code += f"<p>artifact_name: {summary.get('tags.artifact_name', 'No artifact_name found')}</p>"
            html_code += "<br>"

    # Close the HTML code
    html_code += "</div>"
    
    return html_code


## Defining Paths

In [5]:
config_input = 'parameter_config_harmonic.yaml'
config_output = 'model_info_harmonic.yml'

In [6]:
# Define the file paths
# endpoint_url = "https://tenant1.predict.masinst1.ibmmam.com/ibm/modelfactory/service"
endpoint_url = "http://localhost:8000/ibm/modelfactory/service"
data_file_path =  "../data/training.csv"
kpi_event_file_path =  "../config/" + config_input
#kpi_event_file_path =  "../config/parameter_config_harmonic.yaml.yaml"

## POST Response

In [7]:
import requests

# filenames = ["imputate.csv", "KPI.yaml", "sensor_rule.yaml", "timeframe.yaml"]
files = {
    "data_file": ("training.csv", open(data_file_path, 'rb')),
    "kpi_event_file": (config_input, open(kpi_event_file_path, 'rb')),

}

#/ibm/modelfactory/service/recipe/electrical-transformer/anomaly/harmonic/rule
url = endpoint_url + "/recipe/electrical-transformer/anomaly/rule"
post_response = requests.request("POST", url, headers={}, data={}, files=files)

In [8]:
print(url)

http://localhost:8000/ibm/modelfactory/service/recipe/electrical-transformer/anomaly/rule


In [9]:
post_r_json = post_response.json()

anomaly_service_jobId = None

if 'jobId' in post_r_json:
    anomaly_service_jobId = post_r_json['jobId']
    print ('submitted successfully job : ', post_r_json['jobId'])
else:
    print (post_r_json)


{'job_id': 'b5157087-33a7-4c78-8d94-37034a81973c', 'message': 'Job b5157087-33a7-4c78-8d94-37034a81973c was submitted.', 'status': 'INITIALIZING'}


## GET Response  - Keep Refresh it for a while until seeing the STATUS changed to DONE

The status of the job may be running, flagged by INITALIZING or EXECUTING

After a while the model recipe training is complete, and the STATUS changes to DONE

In [10]:
log_url = endpoint_url + "/log/"
job_id = post_r_json['job_id']

In [11]:
print(log_url + job_id)

http://localhost:8000/ibm/modelfactory/service/log/b5157087-33a7-4c78-8d94-37034a81973c


In [14]:
json_data = print_job_details(job_id,endpoint_url)

summary {'run_id': 'b9b0f70280244484b8d55288d5dff230', 'experiment_id': '23', 'status': 'FINISHED', 'artifact_uri': 's3://testdataupload/23/b9b0f70280244484b8d55288d5dff230/artifacts', 'start_time': '2023-11-20T21:56:32.827000+00:00', 'end_time': '2023-11-20T21:56:39.231000+00:00', 'params.storage_path': 'file://./storage/', 'params.data_path': 's3://testdataupload/b5157087-33a7-4c78-8d94-37034a81973c_training.csv', 'params.kpi_event_path': 's3://testdataupload/b5157087-33a7-4c78-8d94-37034a81973c_parameter_config_harmonic.yaml', 'tags.mlflow.project.backend': 'local', 'tags.artifact_name': 'rule_based_harmonic_anomaly', 'tags.mlflow.log-model.history': '[{"run_id": "b9b0f70280244484b8d55288d5dff230", "artifact_path": "rule_based_harmonic_anomaly", "utc_time_created": "2023-11-20 21:56:35.587401", "flavors": {"python_function": {"model_path": "model.pkl", "predict_fn": "predict", "loader_module": "mlflow.sklearn", "python_version": "3.9.2", "env": {"conda": "conda.yaml", "virtualenv": 

### Saving details for deployment

In [15]:
json_data

{'job_id': 'b5157087-33a7-4c78-8d94-37034a81973c',
 'status': 'DONE',
 'summary': {'Deployment Instruction': 'For deployment, pass the parent_run_id as an argument in the deployment recipe',
  'model_uri': 'runs:/b9b0f70280244484b8d55288d5dff230/rule_based_harmonic_anomaly',
  'parent_run_id': 'f0a81683807a46be97ac5c378d612c08'},
 'detailed_summary': [{'run_id': 'b9b0f70280244484b8d55288d5dff230',
   'experiment_id': '23',
   'status': 'FINISHED',
   'artifact_uri': 's3://testdataupload/23/b9b0f70280244484b8d55288d5dff230/artifacts',
   'start_time': '2023-11-20T21:56:32.827000+00:00',
   'end_time': '2023-11-20T21:56:39.231000+00:00',
   'params.storage_path': 'file://./storage/',
   'params.data_path': 's3://testdataupload/b5157087-33a7-4c78-8d94-37034a81973c_training.csv',
   'params.kpi_event_path': 's3://testdataupload/b5157087-33a7-4c78-8d94-37034a81973c_parameter_config_harmonic.yaml',
   'tags.mlflow.project.backend': 'local',
   'tags.artifact_name': 'rule_based_harmonic_anoma

In [16]:
import yaml
if 'status' in json_data:
    if json_data['status'] == 'DONE':
        for summary in json_data["detailed_summary"]:
            if summary['tags.artifact_name']!="":
                model_uri = "runs:/" + summary['run_id'] + "/" + summary['tags.artifact_name']
                s3_uri = summary["artifact_uri"] + "/" + summary['tags.artifact_name']
        output_data = {
            "model_uri" : model_uri,
            "train_job_id" : job_id,
            "s3_uri": s3_uri,
        }
        with open("../config/" + config_output,"w") as file:
            yaml.dump(output_data, file)
        output_data

In [17]:
output_data

{'model_uri': 'runs:/b9b0f70280244484b8d55288d5dff230/rule_based_harmonic_anomaly',
 'train_job_id': 'b5157087-33a7-4c78-8d94-37034a81973c',
 's3_uri': 's3://testdataupload/23/b9b0f70280244484b8d55288d5dff230/artifacts/rule_based_harmonic_anomaly'}